<a href="https://colab.research.google.com/github/ravi-prakash1907/Data-Structures-and-Algo/blob/main/Submissions/Group%20Projects/Project3/English_Sentence_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 3**

## English Sentence Generator

<!--
_**([Final Report Here!!](#))**_
-->


### _Libraries_

In [ ]:
from IPython.display import clear_output
import re

## Input Text Processing

**Steps:**  
1. Get Sentance (Input)  
2. Convert to Lower/Upper Case  
3. Resolve for Abbriviation (ain't, I've, can't...)  
4. Remove Negation  
5. Resolve for Interrogative  
6. Remove Stopping Symbol (. ! ? : ; etc..)
7. Split the Sentance and Store in Dictionary  
  - _Parts of Sentance:_  
    a) Subject  
    b) Helping Verb  
    c) Verb + Object  

_(we can later split **b** in 2-levels and **c** in 2 saperate parts)_

In [ ]:
## returns elements of 'allWords' that are in 'findIn'
def hasWordsFrom(allWords, findIn):
  found = []
  for searched in allWords:
    if searched in findIn:
      found.append(searched)
  if len(found) > 0:
    return found
  return False

## returns indices of elements of 'allWords' that are in 'findIn'
def hasWordsIndexFrom(allWords, findIn):
  found = []
  for searched in allWords:
    for index in reange(len(findIn)):
      if searched == findIn[index]:
        found.append(index)
  if len(found) > 0:
    return found
  return False

## List of words to sentance
def listToSentance(buildingBlocks):
  newSentance = ""
  for word in buildingBlocks:
    newSentance += word
    if newSentance[0] == " ":
      newSentance = newSentance[1:]
    if newSentance[-1] == " ":
      newSentance = newSentance[:-1]
    newSentance += " "
  
  return newSentance[:-1]   # last letter is space (from for-loop)  

### Processing Level 1

1. Taking Input  
2. Converting to Lower Case  
3. Resolving for Abbriviation  
4. Removing Negation  

_(Step 1, 2, 3 and 4)_

In [ ]:
## part of text data cleaner
def toLower(sentance):
  return sentance.lower()


# resolving for abbreviations
def resolveAbbr(sentance):
  if '\'' in sentance:
    splittedSentance = sentance.split(" ")

    for index in range(0, len(splittedSentance)):
      word = splittedSentance[index]

      # abbriviated 'not'
      if word[-3:] == 'n\'t':
        if word == 'can\'t':
          word = 'can not'
        elif word == 'ain\'t':
          word = 'are not'  # or 'is not' too in some cases still it'll work for now
        elif word == 'won\'t':
          word = 'will not'
        elif word == 'shan\'t':
          word = 'shall not'
        else:
          word = word[:-3] + " not"
      
      # abbriviated 'ing'
      if word[-3:] == 'in\'':
        word = word[:-1] + 'g'
      
      # abbriviated helping verbs
      abbrHVerbs = {'m':'am', 's':'is', 're':'are', 'll':'will', 've':'have', 'd':'had'}
      if "'" in word:
        flag = False
        brokenWord = word.split('\'')
        for abbr in list(abbrHVerbs.keys()):
          if  abbr == brokenWord[1]:
            brokenWord[1] = abbrHVerbs[abbr]
            flag = True

        if flag:
          word = listToSentance(brokenWord)
        else:
          word = brokenWord[0] + '\'' + brokenWord[1]
      
      splittedSentance[index] = word

    resolvedTense = listToSentance(splittedSentance)
    return resolvedTense

  return sentance


## Later we'll read sntances from some other source(s).
def readSentance():
  textData = input("Enter a sentance: ")
  return textData


## Removing Negation
def negationRemover(sentance):
  if 'not' in sentance:
    begIndex = sentance.find("not")
    sentance = sentance[:begIndex-1] + sentance[begIndex+3:]
    sentance = negationRemover(sentance)
  return sentance


### Processing Level 2

1. Resolving for Interrogatives  
2. Remove Stopping Symbol (. ! ? : ; etc..)

_(Step 5)_

In [ ]:
## Yet to be implemented
def removeQuestion(sentance):
  print("Interrogative to normal sentance converter is yet to be impleented!\n")
  return sentance

def removeStops(sentance):
  if not sentance[-1].isalpha() and not sentance[-1].isdigit():
    sentance = removeStops(sentance[:-1])
  return sentance

### Processing Level 3

**Initial Splitting** _(if consists Helping Verb)_  

**Parts of Sentance:**  
  - Subject + Helping Verb  
  - Verb + Object  


_(Step 6.1)_

In [ ]:
## Building Blocks

# splits as:  ['sub + HV', 'verb + obj']
def splitOnHelpingVerb(sentance):
  ## H.V. to split
  firstPriority = ['is', 'am', 'are', 'was', 'were']
  secondPriority = ['been', 'be']
  thirdPriority = ['have']
  forthPriority = ['shall', 'will', 'may', 'can', 'must', 'should', 'would', 'might', 'could', 'do', 'does', 'did', 'has', 'had']

  helpingVerbs = [firstPriority, secondPriority, thirdPriority, forthPriority]
  initSplit = []
  splitted = False
  
  for priorityList in helpingVerbs:
    for helpingVerb in priorityList:
      if helpingVerb in sentance:
        initSplit = sentance.split(helpingVerb)
        initSplit[0] += helpingVerb
        splitted = True
        break
    if splitted:
      break
  
  return initSplit


def consistsHelpingVerb(sentance = "he goes"):
  ## Helping Verbs List
  helpingVerbs = ['is', 'am', 'are', 'was', 'were', 'been', 'be', 'have', 'shall', 'will', 'may', 'can', 'must', 'should', 'would', 'might', 'could', 'do', 'does', 'did', 'has', 'had']
  if hasWordsFrom(helpingVerbs, sentance.split(" ")):
    splittedForm = splitOnHelpingVerb(sentance)
    return splittedForm,True
  else:
    HV = ''
    splittedForm = splitSimple(sentance)
    splittedForm.insert(1,HV)
    return splittedForm,False

#######################################

## he is a bad boy, although he is ready for singing for now as it is already 10 o'clock
consistsHelpingVerb()


(['he', '', 'goes'], False)

### Processing Level 4

**Secondary Splitting**  

_Parts of **Subject + Helping Verb** :_  
  - Subject  
  - Helping Verb  

_(Step 6.2)_

In [ ]:
## split ["Sub", "helpingVerb / verb / verb+obj"]
def splitFromBeg(sentance, consistsHV = False):  # rename 'consistsHV'  --->  'helpingVerbs'
  if len(sentance.split(" ")) == 2:
    return sentance.split(" ")
  
  elif len(sentance.split(" ")) > 2 and consistsHV:

    #############################################################################
    ####  this list can be feeded as input to the 'consistsHV'
    firstPriority = ['is', 'am', 'are', 'was', 'were']
    secondPriority = ['been', 'be']
    thirdPriority = ['have']
    forthPriority = ['shall', 'will', 'may', 'can', 'must', 'should', 'would', 'might', 'could', 'do', 'does', 'did', 'has', 'had']
    
    helpingVerbs = [forthPriority, thirdPriority, secondPriority, firstPriority]
    #helpingVerbs = [secondPriority, thirdPriority, forthPriority, firstPriority]
    #############################################################################

    begSplitted = []
    splitted = False
    
    for priorityList in helpingVerbs:
      for helpingVerb in priorityList:
        if helpingVerb in sentance:
          begSplitted = sentance.split(helpingVerb)
          begSplitted[1] = helpingVerb + begSplitted[1]
          splitted = True
          break
      if splitted:
        break
    return begSplitted
  else:
    wordList = sentance.split(" ")
    print("Yet to be implemented for sentences without HV. (like: 'he goes.')\nReturning word-list")
    return wordList

### Processing Level 5

**Secondary Splitting** _(without HV)_  

_Parts of **Subject + [Verb + Object**] :_  
  - Subject
  - Verb  
  - Object  

_(Step 6.5)_

In [ ]:
def splitSimple(sentance = "He works"):
  wordList = sentance.split(" ")
  totalWords = len(wordList)
  
  if totalWords <= 2:
    return wordList
  else:
    Subject = Verb = Object = ''

    # checking for Simple Past
    if sentance[-2:] == 'ed' or 'ed ' in sentance:
      for word in wordList:
        if 'ed' in word or 'ed ' in word:
          Subject = listToSentance(wordList[:wordList.index(word)])
          Verb = word
          Object = listToSentance(wordList[wordList.index(word)+1:])
          break
      
    # checking for Simple Present (mainly)
    else:
      articles = ['a', 'an', 'the']
      pluralPronouns = ['some', 'any', 'these', 'those', 'them', 'they', 'you', 'i', 'we']
      singlelPronouns = ['this', 'that', 'it']  # Tackel more at :   http://grammar.ccc.commnet.edu/grammar/determiners/determiners.htm#:~:text=These%20categories%20of%20determiners%20are,any%2C%20etc.)%3B%20and%20demonstrative

      sEsEndingWords = []
      for word in wordList:
        if (word[-2:] == 'es') or (word[-1] == 's' and not (word[-2:] == 'us' or word[-2:] == 'ss')):
          sEsEndingWords.append(word)

      usedArticles = hasWordsFrom(articles, wordList)

      # if consisits articles
      if usedArticles:
        if wordList[0] in articles and len(sEsEndingWords) == 1:
          wordList.remove(wordList[0])
          totalWords -= 1
          subEndIndex = wordList.index(sEsEndingWords[0])
          if sEsEndingWords[0] == wordList[0]:
            Subject = listToSentance(wordList[:subEndIndex+1])
            Verb = listToSentance(wordList[subEndIndex+1:])    # includes object
            Object = Verb # includes verb
          else:
            Subject = listToSentance(wordList[:subEndIndex])
            Verb = sEsEndingWords[0]
            Object = listToSentance(wordList[subEndIndex:])
        else:
          objBegIndex = wordList.index(usedArticles[0])
          Subject = listToSentance(wordList[:objBegIndex-1])
          Verb = wordList[objBegIndex-1]
          Object = listToSentance(wordList[objBegIndex:])
      
      else:
        if len(sEsEndingWords) == 0:
          pluralSub = hasWordsFrom(pluralPronouns, wordList)
          if pluralSub:
            subEndIndex = wordList.index(pluralSub[0]) + 1
          elif ' and ' in sentance:
            subEndIndex = wordList.index('and') + 1
          else:
            subEndIndex = 1
          Subject = listToSentance(wordList[:subEndIndex])
          Verb = listToSentance(wordList[subEndIndex:subEndIndex+1])    # may includes object
          Object = listToSentance(wordList[subEndIndex+1:])  # may includes verb
        else:
          if wordList.index(sEsEndingWords[0]) == 0:
            Subject = listToSentance(wordList[:1])
            Verb = listToSentance(wordList[1:2])    # includes object
            Object = listToSentance(wordList[2:]) # includes verb
          else:
            singleSub = hasWordsFrom(singlelPronouns, wordList)
            if singleSub:
              subEndIndex = wordList.index(singleSub[0]) + 1
            else:
              subEndIndex = 1
            Subject = listToSentance(wordList[:subEndIndex])
            Verb = listToSentance(wordList[wordList.index(sEsEndingWords[0]):wordList.index(sEsEndingWords[0])+1])

            if len(sEsEndingWords) == 1 and wordList.index(sEsEndingWords[0]) == len(wordList)-1:
              Object = ''
            if len(sEsEndingWords) > 1 and wordList.index(sEsEndingWords[0]) not in [0, len(wordList)-1]:
              Object = listToSentance(wordList[wordList.index(sEsEndingWords[0])+1:])
    
    return [Subject, Verb, Object]






In [ ]:
splitSimple()

['He', 'works']

## **Driver (for testing) Function**

In [ ]:
def driverFun():
  ## Processing Level 1
  aSentance = readSentance()
  print()
  
  oneCase = toLower(aSentance)
  abbrResolved = resolveAbbr(oneCase)
  positiveTense = negationRemover(abbrResolved)

  ## Processing Level 2
  nonQuestion = removeQuestion(positiveTense)
  cleanSentance = removeStops(nonQuestion)
  #print("Clean sentance: ",cleanSentance,"\n")

  ## Processing Level 3
  threeOrFourParts,flag = consistsHelpingVerb(cleanSentance)
  #print(threeOrFourParts)

  ## Processing Level 4
  if flag:
    sentanceBeg = splitFromBeg(threeOrFourParts[0], True) # true means tense has some HV
    fullSentance = []
    fullSentance.extend(sentanceBeg)
    fullSentance.append(twoParts[1])
  else:
    fullSentance = threeOrFourParts
  
  print(fullSentance)


##############

driverFun()

##  "Charlie hasn't been competin' in flute competitions recently!!!"

Enter a sentance: Every Monday they eat spaghetti for dinner.

Interrogative to normal sentance converter is yet to be impleented!

['every monday they ', '', 'eat ', 'spaghetti for dinner ']


In [ ]:

def consistsHelpingVerb(sentance = "he goes"):
  ## Helping Verbs List
  helpingVerbs = ['is', 'am', 'are', 'was', 'were', 'been', 'be', 'have', 'shall', 'will', 'may', 'can', 'must', 'should', 'would', 'might', 'could', 'do', 'does', 'did', 'has', 'had']
  if hasWordsFrom(helpingVerbs, sentance.split(" ")):
    print("checking in hv")
    splittedForm = splitOnHelpingVerb(sentance)
    return splittedForm,True
  else:
    print("checking in non-hv")
    HV = ''
    splittedForm = splitSimple(sentance)
    splittedForm.insert(1,HV)
    return splittedForm,False


consistsHelpingVerb("every monday they eat spaghetti for dinner")

checking in non-hv


(['every monday they ', '', 'eat ', 'spaghetti for dinner '], False)

In [ ]:
x = [1,2,3,4].insert(2,200)
print(x)

None


## Storing in Tree

**Steps:**  

# **Users' Zone**

## Main-Menu

In [ ]:
def menu():
  # Printing Menu
  print("\nMain Menu:")
  print("""----------
        \n1. Load Raw Images  \n2. Create New Album  \n3. Grab n-th Image \n4. Print Loaded Images \n5. Print Newly Created Album \n
        \n** Any other key to exit!!\n
        """)
  return input("Enter your choice: ")

## Main-Function


In [ ]:
### To be Implemented

if __name__ == "__main__":
  
  ## Showing Menu

  while (True): 
    clear_output()
    #choice = menu()
    
    print("Yet to be implemented!\nMeet Soon!!!")
    break

Yet to be implemented!
Meet Soon!!!
